# DMRG code
this is a dmrg to get the ground state energy from tenpy package 

In [77]:
import time as tt
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import tenpy as tp
# tp.tools.misc.setup_logging(to_stdout="INFO")
# from numba import njit, prange
# from numba import njit, prange

In [ ]:
"Isisng model DMRG"
import logging
logging.disable(logging.INFO)


L = 10
gg, jj = 0.5, 1.0
model_params = {
    'J': jj , 'g': gg,  # critical
    'L': L,
    'bc_x': 'open',
    'bc_MPS': 'finite',
}
dmrg_params = {
    'mixer': False,  # setting this to True helps to escape local minima
    'max_E_err': 1.e-14,
    'trunc_params': {
        'chi_max': 100,
        'svd_min': 1.e-14,
    },
    'verbose': True,
    'combine': True,
};
M = tp.models.tf_ising.TFIChain(model_params)
# print("*********")
# print( M.lat.pairs['nearest_neighbors'][0][1]  )
# print(M.lat.)
# print("*********")
psi = tp.networks.mps.MPS.from_lat_product_state(M.lat, [['up']])
eng = tp.algorithms.dmrg.TwoSiteDMRGEngine(psi, M, dmrg_params);
E, psi = eng.run();
print("")
print("")
print("ground state energy = ", E)


*********
*********
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))



ground state energy =  -9.765503957927175


# the code that did NOT worked !!!

In [ ]:

L = 10
Vv, Jj = 0.3, 0.3
model_paramss = {
    'Jxx': -2*Jj , 'Jz': Vv/8, 'hz':-Vv/4,
    'L': L,
    'bc_x': 'open',
    'bc_MPS': 'finite',
}

dmrg_params = {
    'mixer': False,  # setting this to True helps to escape local minima
    'max_E_err': 1.e-14,
    'trunc_params': {
        'chi_max': 100,
        'svd_min': 1.e-14,
    },
    'verbose': True,
    'combine': True,
};


M = tp.models.xxz_chain.XXZChain(model_paramss);
# M.add_onsite_term(-Vv/4, 0, 'Sz')
# M.add_onsite_term(-Vv/4, L-1, 'Sz')
# for indx in range(L):
#     M.add_onsite_term(Vv/8, indx,'Id')

psi = tp.networks.mps.MPS.from_lat_product_state(M.lat, [['up'],['down']])
eng = tp.algorithms.dmrg.TwoSiteDMRGEngine(psi, M, dmrg_params);
E, psi = eng.run();
print("")
print("")
print("ground state energy = ", E + Vv*(L-1)/8 )
# print("constant energy = ", Vv*(L-1)/8 )
Z = psi.expectation_value('Sz')
print("total magnetization =", np.sum(Z))

  warnings.warn(msg.format(keys=sorted(unused), name=self.name))



AttributeError: 'XXZChain' object has no attribute 'init_H_from_terms'

In [21]:
print("ground state energy = ", E + Vv*(L-1)/8)
# print("constant energy = ", Vv*(L-1)/8 )

# Z = psi.expectation_value('Sz')
# print("total magnetization =", np.sum(Z))


ground state energy =  -1.5104829700472446


# the code that worked !!!!

In [ ]:
import tenpy
from tenpy.networks.site import SpinHalfSite
from tenpy.models.spins import SpinModel #,SpinHalfSite
from tenpy.algorithms import dmrg
from tenpy.networks.mps import MPS
import logging
logging.disable(logging.INFO)


L = 18  # Number of sites
Jj = 1.0  
V = 0.4  
h_boundary = V/4 #boundary magnetic field strength.
model_params = {
    'S':0.5,
    'Jx': 2*Jj, 'Jy': 2*Jj, 'Jz': V/2,
    'hz': -V/1,
    'L': L,
    'conserve':'Sz',
    'bc_x': 'open',
    'bc_MPS': 'finite',
}

# Define the spin-1/2 site
site = SpinHalfSite()

# Create the spin model
model = SpinModel(model_params)
# Add boundary terms
model.add_onsite_term(h_boundary, 0, "Sz") #left boundary
model.add_onsite_term(h_boundary, L-1, "Sz") #right boundary
model.init_H_from_terms()


# Add Heisenberg interactions
# model.add_coupling_term(J, 0, 1, "Sz", "Sz")
# for i in range(L - 1):
#     model.add_coupling_term(0.0*J, i, i + 1, "Sx", "Sx")
#     model.add_coupling_term(0.0*J, i, i + 1, "Sy", "Sy")

# Add on-site magnetic field
# for i in range(L):
#     model.add_onsite_term(V/9, i, "Sz")
# model.init_H_from_terms()



psi = tp.networks.mps.MPS.from_lat_product_state(model.lat, [['up'],['down']])


# DMRG parameters
dmrg_params = {
    'mixer': False,  # setting this to True helps to escape local minima
    'max_E_err': 1.e-16,
    'trunc_params': {
        'chi_max': 100,
        'svd_min': 1.e-16,
    },
    'verbose': True,
    'combine': True,
};

# Run DMRG
results = dmrg.run(psi, model, dmrg_params)

# Print results
print("DMRG results:", results['E'])
print("DMRG + constant energy = ", results['E'] + V*(L-1)/8 )

  model.add_onsite_term(h_boundary, 0, "Sz") #left boundary

  model.add_onsite_term(h_boundary, L-1, "Sz") #right boundary

  warnings.warn(msg.format(keys=sorted(unused), name=self.name))

DMRG results: -11.493536631155479
DMRG + constant energy =  -10.643536631155479


In [65]:
from tenpy.models.lattice import Site, Chain
from tenpy.models.model import CouplingModel, NearestNeighborModel, MPOModel, CouplingMPOModel
from tenpy.linalg import np_conserved as npc
from tenpy.tools.params import asConfig
from tenpy.networks.site import SpinHalfSite

In [ ]:
class SpinFermionChain(CouplingMPOModel, NearestNeighborModel, MPOModel):
    default_lattice = "Chain"
    force_default_lattice = True

    def init_sites(self, model_params):
        sort_charge = model_params.get('sort_charge', True, bool)
        return SpinHalfSite(conserve=None, sort_charge=sort_charge)  # use predefined Site

    def init_terms(self, model_params):
        # read out parameters
        J = model_params.get('J', 1., 'real_or_array')
        V = model_params.get('V', 1., 'real_or_array')
        # Jzz = model_params.get('Jzz', 0., 'real_or_array')
        # add terms
        for u in range(len(self.lat.unit_cell)):
            # print("**********************")
            # print(u,",",",",u)
            # print("**********************")
            self.add_onsite(+V/4, u, 'Sz')
            # self.add_onsite(-hx, u, 'Sx')
        
        self.add_onsite(-V/8, 0, 'Sz')
        self.add_onsite(-V/8, len(self.lat.unit_cell)-1, 'Sz')
        
        # for u1, u2, dx in self.lat.pairs['nearest_neighbors']:
        # for u1, u2, dx in self.lat.nearest_neighbors:
        #     print("**********************")
        #     print(u1,",",u2,",",dx)
        #     print("**********************")
        #     self.add_coupling(-J, u1, 'Sp', u2, 'Sm', dx)
        #     self.add_coupling(-J, u1, 'Sm', u2, 'Sp', dx)
        #     self.add_coupling(V/8, u1, 'Sz', u2, 'Sz', dx)
            

In [116]:
L = 8
Vv, Jj = 0.3, 1.0

model_paramss = {
    'J': -1.0 , 'V': 0.1,
    'L': L,
    'bc_x': 'periodic',
    'bc_MPS': 'finite',
}

dmrg_params = {
    'mixer': True,  # setting this to True helps to escape local minima
    'max_E_err': 1.e-10,
    'trunc_params': {
        'chi_max': 100,
        'svd_min': 1.e-10,
    },
    # 'verbose': True,
    'combine': True,
};


M = SpinFermionChain(model_paramss);
psi = tp.networks.mps.MPS.from_lat_product_state(M.lat, [['up'],['down']])
# eng = tp.algorithms.dmrg.TwoSiteDMRGEngine(psi, M, dmrg_params);
# E, psi = eng.run();


# psi = tp.networks.mps.MPS.from_lat_product_state(M.lat, [['up'],['down']])
# eng = tp.algorithms.dmrg.TwoSiteDMRGEngine(psi, M, dmrg_params);

INFO    : SpinFermionChain: reading 'bc_MPS'='finite'
INFO    : SpinFermionChain: reading 'bc_x'='periodic'
INFO    : SpinFermionChain: reading 'L'=8
INFO    : SpinFermionChain: reading 'J'=-1.0
INFO    : SpinFermionChain: reading 'V'=0.1


IndexError: list index out of range